# Predicting Brooklyn House Prices

*Runtime - 2 hours. An entire hour goes in one cell.*

The notebook below shows a project completed during Imperial College London's inaugral 'AI Hack' in November 2018. The challenge was to, given a dataset with a number of samples and features, predict current house prices for the Brooklyn area.
A number of data cleaning techniques were used as much of the data was unrecorded, or unusable (e.g. having a year of sale as 0), which had to be accounted for.
Once data cleaning was done, the non-numerical data (e.g. categorical variables) were removed, and of the remaining variables, even more were discarded based on the author's discretion. 

Finally, with a usable dataset, a number of machine learning techniques were used to estimate house prices on a validation set, and the performance of each technique was measured by the R^2 value between the predicted and actual sale price for each sample in the validation dataset. The best performing method was the RandomForestRegressor model provided in the sklearn library, with an R$^2$ value just under 0.8, and outperformed all other models comfortably.

# Loading the data

First ensure we're in the right directory. Then import all relevant modules, after which, load all the data.

In [ ]:
cd /Users/arohan/Documents/CODING/AI_Hack/brooklyn

In [ ]:
ls #checking we're in the right folder

In [ ]:
#import pandas for dataframe processing
import pandas as pd

#import numpy
import numpy as np
from numpy import array
from numpy import argmax

#import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

#set warnings off
import warnings
warnings.filterwarnings('ignore')

#import sklearn for the ML
import sklearn
from sklearn.preprocessing import Imputer
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# importing the raw data from the .csv file
df_trainvalid_raw = pd.read_csv("brooklyntrainset.csv") #training dataset, to be split into train and test
df_testraw = pd.read_csv("brooklyntestset.csv") #validation dataset
pd.set_option('display.max_columns', 111) #viewing preference, max columns visible
print(df_trainvalid_raw.shape)
print(df_testraw.shape)


Notice that they have different numbers of rows - expected, as obviously the validation set isn't going to be as large as the training set. However, the training set has one more column than the test set, why? And which column is that?

In [ ]:
traval_cols = df_trainvalid_raw.columns.values
test_cols = df_testraw.columns.values

#print the col which isn't in test_cols
for name in traval_cols:
    if name not in test_cols:
        print(name)
    
#df_trainvalid_raw['sale_price'][df_trainvalid_raw['sale_price'] == 0]

The extra column is the dependent variable/label/what we're trying to predict, sale_price. Looking at the dataset shows us that a lot of rows in the training set have a sale_price of 0 (uncomment the bottom line above) - this is useless, so get rid of all rows that have a sale_price of 0:

# Dealing with sale_price = 0 and NaNs

In [ ]:
#Combine the training/validation and test data
df_all_data_raw = pd.concat([df_trainvalid_raw, df_testraw],ignore_index=True)
print("Number of all_data rows with 'sale_price=0' included: ",len(df_all_data_raw))
df_all_data = df_all_data_raw.drop(df_all_data_raw.loc[df_all_data_raw['sale_price'] == 0].index,axis=0)
print("Number of all_data rows after 'sale_price=0' rows dropped: ", len(df_all_data)) 

n_data = len(df_all_data) #number of data points for training/testing
n_data_valid = len(df_testraw) #number of test points for validation (where sale_price = np.NaN)
n_data_train = n_data - n_data_valid
#df_all_data.head()    

Upon further inspection of the data, we find there are a lot of **features** (i.e. columns) with many unrecorded values - NaNs. These are not useful so we get rid of them:

In [ ]:
#all_na is a df with where every value is mapped to true if null and false otherwise
#Summing it sums up all the true/falses in each col, and then it's turned into a %

df_all_na=(df_all_data.isnull().sum() / len(df_all_data)) *100 #all_na is 112 long
len(df_all_na[df_all_na==0]) #17 columns with no NAs

#Drop the (17) columns that have no NAs, below:
df_all_na=df_all_na.drop(df_all_na[df_all_na==0].index).sort_values(ascending=False)

#Now we have 112 - 17 = 95 columns in the all_na array
#Getting rid of 20 columns with the most NAs, from all_data
df_data_clean = df_all_data.drop(df_all_na[0:20].index,axis=1) 

#This should leave us with 112-20 = 92 columns. Check using len(list(all_data_clean))

We've dropped the 20 columns with the most NaNs - however, there are still many NaN values in other columns. We deal with this by 'imputing' the data - replacing the NaN value with a suitable alternative. For some columns, this means replacing NaNs with the average value from the rest of the data set. 

Also notice: because the **label** (Y value) is ``sale_price``, which is a float, it is in the df_num2_withy dataframe below. We create this so that we can drop the ``sale_price`` column from it, so that when imputing missing values later on we don't fill in values for sale_price - this would ruin the model.

In [ ]:
#Split the data into different variable types
#integer type columns
df_num1 = df_data_clean[df_data_clean.select_dtypes(include=['int64']).columns]
num1_colnames = list(df_num1) #need to save column names for later

#float type columns
df_num2_withy = df_data_clean[df_data_clean.select_dtypes(include=['float64']).columns]
df_num2 = all_data_num2_withy.drop('sale_price',axis = 'columns',inplace=False)
num2_colnames = list(df_num2) #need to save column names for later

#object type columns
all_data_obj = df_data_clean[df_data_clean.select_dtypes(include=['object']).columns]

# Imputing Data

Now we create the imputer, and impute all the NaN values in all three dataframes that we have made.

The imputer function works based on numerical values (as it imputes missing values with the mean) therefore it can't be used for the object columns, and we take a different approach for those. First, drop any columns that seem inconsequential to the model and that we can't hope to take any data from. Then just replace any NaN values with an empty string, ' '.

In [ ]:
Imputer() #create imputer object

In [ ]:
imp = Imputer(axis=0,copy=True,missing_values=(np.nan),strategy='mean')

imp.fit(df_num1)
all_data_num1 = imp.transform(df_num1)
imp.fit(df_num2)
all_data_num2 = imp.transform(all_data_num2)

#Drop useless object columns 
drop_objs = ['Address','OwnerName','Borough','Version','address','building_class_category','neighborhood']
all_data_obj_dropped = all_data_obj.drop(drop_objs,axis = 'columns',inplace=False)

#Replace remaining NaNs with empty strings
all_data_obj_dropped.replace(np.nan,'',regex=True, inplace=True)


# Dealing with Categorical Data

Now usually, to convert the remaining categorical variables into useful data points, we have to 'one hot encode' them. This means first assign a numerical value for each categorical variable. This works in some cases but mostly it implies that certain valeus are inherently better than others - which is often untrue for categorical values.

So, to compensate for this, one hot encoding takes place. Assuming there are 5 possible categorical values (e.g. colours red, blue, green, yellow, pink) and there are 300 total data points (e.g. 300 people surveyed on their favourite colour), then each person's colour choice becomes an array of length 5, with one '1' corresponding to their favourite colour and the rest of the array filled with zeros.

Example:
    Tony - red,
    Steve - blue,
    Bruce - green,
would become:
    Tony - [1,0,0],
    Steve - [0,1,0],
    Bruce - [0,0,1]
    
Ideally this is what we should do with all the categorical variables. However if we do this, our dataset just becomes stupidly big. For example, there are over 100 types of 'FireComp', which is just ONE variable, and there are 12 of these variables, and 250,000 data points for each. Let's leave that for now and stick to the numerical data that we have.

# Deleting Sparse rows and columns

Another thing we found from the data is there are many 'sparse' rows and columns, AKA a lot of rows and columns mostly filled with zeros. These haven't been picked up by the imputer, as it only imputes values that are NaNs (which zeros don't count as). We make the decision to get rid of all rows with more than a certain threshold of zeros.

This has to be done using arrays, as the data is now in list/array form after the imputing stage:

In [ ]:
zero_thresh = 0.25 #if more than this fraction of values are zero, drop the row
count = [] #to show how many rows have been dropped
all_data_nums=[] #the new list we store our data in

#Concatenate Y row with rest of data
Y = np.reshape(np.array(all_data_num2_withy['sale_price']),[len(all_data_num1),1])
temp_nums=np.concatenate((all_data_num1,all_data_num2,Y),axis=1) #imputed integer and float columns


for i in range(len(temp_nums)):
    if(np.count_nonzero(temp_nums[i] == 0) < (zero_thresh * (len(num1_colnames)+len(num2_colnames)))):
        all_data_nums.append(temp_nums[i]) #append if few enough zeros
    else:
        count.append(np.count_nonzero(temp_nums[i]==0)) #append if too many zeros

n_zerorows_dropped = len(count)
print(n_zerorows_dropped,"out of",len(temp_nums),"rows dropped due to too many zeros.")

The next step is to drop any *columns* with too many zeros. This is easiest to do when the data is converted back into dataframe format, so first convert the data into a dataframe:

In [ ]:
df_all_data_nums = pd.DataFrame(all_data_nums,columns = num1_colnames+num2_colnames+['sale_price'])
df_all_data_nums.head()

Then find the number of zeros in each column as a % of the total number of values in the column. Then remove all columns with less than a certain threshold of non-zero values:

In [ ]:
show_na = (df_all_data.astype(bool).sum(axis=0) / len(df_all_data))*100

#drops all columns with less than threshold% non-zero values
threshold = 40
sorted_na = show_na[show_na < threshold].sort_values(ascending=True)
all_data_with_outliers = df_all_data.drop(sorted_na.index,axis=1)

Remember at the start when we joined trainvalid_raw and test_raw data together?
*(We did this so we could impute data for both of them at the same time)*

Now, we should split the data into training/validation and test again. Then later, we can split the training/validation into training and validation data. We'll train our model on the training data and test it on the validation data. For the test data, we don't actually have the answers so we'll just leave that alone!

# Split all data into training/validation ('model') and test data

In [ ]:
n_sale_nans = all_data_with_outliers['sale_price'].isnull().sum()
n_data_points = len(all_data_with_outliers)
n_usable = n_data_points - n_sale_nans
print(n_usable)
print(n_sale_nans)


#Now we know that all_data_with_outliers is 196871 rows x 59 columns. We also know the first 167882 rows are training data and the next 28989 (n_sale_nans) rows are test data.
#So let's split them up accordingly:

#model refers to training and validation data together
model_data = pd.DataFrame(all_data_with_outliers.iloc[:n_usable])
test_data = pd.DataFrame(all_data_with_outliers.iloc[n_usable:])
print('Number of features:',len(list(model_data)))

As we can see, there are still 59 columns worth of indicators - not all of these will be useful. Although we have got rid of columns with too many NaNs and zeros, we now need to see what they actually mean and if they're useful or not. Here goes:

In [ ]:
print(list(model_data))

# Manually remove unnecessary features

In [ ]:
#model_data['Unnamed: 0'] #what even is that, nope
#model_data['X'] #what even is that, nope
model_data['block'] #no NaNs, no zeros. Keep
#model_data['borough'] #every borough is 3. nope
#model_data['lot'] #lot of exceptions etc and not categorical or continuous data, nope
model_data['residential_units'] #residential units in a tax lot. Has one crazy big outlier.
model_data['tax_class_at_sale'] #categorical. Get rid of all rows with tax classes 3 and 4, non residential
model_data['total_units'] #total units at the property. YES
model_data['year_built'] #watch out for zeros. But Yes.
model_data['year_of_sale'] #no zeros. YES
#model_data['zip_code'] #same as tax lot. No thanks
#model_data['AreaSource'] #to do with source files for tax lot allocation. nope
model_data['AssessLand'] #assessed value of the land if vacant. YES
model_data['AssessTot'] #assessed value for the total year. YES
#model_data['BBL'] #borough, block and lot in one. nope
model_data['BldgArea'] #total gross area in sqft
model_data['BldgDepth'] #keep, only couple of horizontal outliers
model_data['BldgFront'] #keep, only couple of horizontal outliers
#model_data[BoroCode] #borough, nope
model_data['BsmtCode'] #continuous, standard of basement, may as well
#model_data[BuiltFAR]  #floor area as ratio of tax lot, no need (got BldgArea)
#model_data['CB2010'] #census block, nope
#model_data['CD'] #community district, categorical data, nope
#model_data['CT2010'] #census tract, nope
#model_data['Council'] #similar to borough, all refer to Brooklyn obvs, nope
#model_data['ExemptLand'] #Land Exempt from tax. Nope
#model_data[ExemptTot'] #Something else Exempt from tax. Nope
#model_data['FacilFAR'] #max allowable floor area ratio? nope
#model_data['HealthArea'] #Health Area code - categorical
#model_data['HealthCent'] #Health centre code - categorical
#model_data['LandUse'] #type of building - useful to get rid of non residential buildings. 
#model_data['LotArea'] #tax lot area
#model_data['LotDepth'] #tax lot depth
#model_data['LotFront'] #tax lot frontage
#model_data['LotType'] #tax lot type 
#model_data['NumBldgs'] #num of bldgs in the tax lot
#model_data['NumFloors'] #num of floors of highest building in tax lot
#model_data['PLUTOMapID'] #PLUTO database ID of some sort
#model_data['PolicePrct'] #police precinct
#model_data['ProxCode'] #categorical data, proximity to closest house. Doesn't make a diff
#model_data['ResArea'] #residential floor area. Lot of zeros. nope
#model_data['ResidFAR'] #residential floor area ratio to tax lot area
#model_data['SHAPE_Area'] #not sure? 
#model_data['SHAPE_Leng'] #not sure?
#model_data['SanitBoro'] #sanitation borough code for tax lot
#model_data['SanitDistr'] #sanitation district code for tax lot
#model_data['SchoolDist'] #school district for code tax lot
#model_data['TaxMap'] #tax map code for tax block and lot
#model_data['Tract2010'] #another census tract code
#model_data['UnitsRes'] #residential units in the tax lot, 1% values are 0 - already have it
#model_data['UnitsTotal'] #all units in the tax lot. Not sure how it'll be useful.
model_data['XCoord'] #location of the lot. Remove zeros and YES
model_data['YCoord'] #location of the lot. Remove zeros and YES
#model_data['YearAlter1'] #lots of exceptions, but year the buiding was altered. nope
#model_data['YearBuilt'] #year built - remove zeros!
#model_data['ZipCode'] #zip code for tax lot - categorical
#model_data['gross_sqft'] #total gross sqft - same as BldgArea pmuch? Remove those that aren't same?
model_data['land_sqft'] #land in sqft. Yep

print()

The commented out lines above are columns we want to drop, and the reason for it is listed next to it. Based on our intuition and knowledge, we have reduced 59 features to 15 useful ones (though we won't keep every one of those in by the end either).

In [ ]:
required_columns = ['block','residential_units','total_units','year_built','year_of_sale','AssessLand','AssessTot','BldgArea','BldgDepth','BldgFront','BsmtCode','XCoord','YCoord','land_sqft','sale_price']
model_data_stepone = pd.DataFrame()
    
for col_id in list(model_data):
    if col_id in required_columns:
        model_data_stepone[col_id] = model_data[col_id]

#use tax_class_at_sale to remove all non residential buildings (they have a tax class of 4)
model_data_steptwo = model_data_stepone.drop(model_data['tax_class_at_sale'][model_data['tax_class_at_sale'] == 4].index)
model_data_steptwo.head()

# Dealing with Outliers by feature

Now, we need to assess the remaining variables - some of them have outliers, some of them have sparse data, and some still have zeros. By plotting the data we can see where exceptional outliers lie, and get rid of these values for our model. NB: this means our predictions are only valid for a certain range of parameters (which we can specify later).

Check the rows on a case by case basis:

(You can plot each param with/without their constaints to see the effect of outliers)

In [ ]:
#land_sqft - get rid of values > 150000
#YES year built - impute with median
#not residential_units - get rid of values > 100
#not tot_units - get rid of values > 100
#not BsmtCode, its legit
#not BldgFront - get rid of values > 200
#BldgDepth - get rid of values > 300
#YES XCoord - median the zeros
#YES YCoord - median the zeros
#block - leave as is
#year of sale - leave as is
#AssessLand - get rid of values > 3,000,000
#AssessTot - leave as is
#BldgArea - leave as is
#sale_price - get rid of values > 4,000,000 

param = model_data_steptwo['BldgFront'][model_data_steptwo['BldgFront'] < 200]
print('Rows that will be lost:',(len(model_data_steptwo) - len(param)))
#plt.scatter(param.loc[:].index,param.loc[:])
#plt.show()

Now, we need to make a new dataframe with all values that don't fit the constraints to be taken out. There are many approaches for this but we choose to make a new dataframe after checking each row if it fits all the constraints. 

(To be honest this code is quite slow and can be improved, but for now we'll keep it at this. It prints out the row number every 5000 rows, and there are about 160000 rows altogether. It takes about an hour to run the code block once!)

In [ ]:
model_data_stepthree = pd.DataFrame(columns=model_data_steptwo.columns.values)

print(len(model_data_steptwo))
#this is gona take a while.
for index, row in model_data_steptwo.iterrows():
    if (index%5000 == 0):
        print(index)
    if (row['residential_units'] < 100):
        if (row['total_units'] < 100):
            if (row['BldgFront'] < 200):
                if (row['BldgDepth'] < 300):
                    if (row['AssessLand'] < 3000000):
                        if (row['sale_price'] < 4000000):
                            if (row['land_sqft'] < 150000):
                                model_data_stepthree.loc[index] = row

Now we need to impute zero values in year_built, land_sqft, XCoord and YCoord with the median value from the dataset.

In [ ]:
med_year = model_data_stepthree['year_built'].median()
med_XCoord = model_data_stepthree['XCoord'].median()
med_YCoord = model_data_stepthree['YCoord'].median()
med_landsqft = model_data_stepthree['land_sqft'].median()

model_data_stepfour = model_data_stepthree

model_data_stepfour['year_built'].replace(to_replace=0.0,value=med_year,inplace=True)
model_data_stepfour['XCoord'].replace(to_replace=0.0,value=med_XCoord,inplace=True)
model_data_stepfour['YCoord'].replace(to_replace=0.0,value=med_YCoord,inplace=True)
model_data_stepfour['land_sqft'].replace(to_replace=0.0,value=med_landsqft,inplace=True)

model_data_stepfour.head()

# Split data into training and validation

Finally, split the training data into training and checking model vailidity - we don't have any sale_price values for the test data, so can't internally test our model.

model_data is all the data we can use for our training, which we will break up into train/test. The 'test' data in this case is used for internal testing, and the 'valid' data is validation data - it doesn't have values for sale price, so it's not really useful to us I guess.

In [ ]:
train = model_data_stepfour.sample(frac = 0.8)
valid = model_data_stepfour.sample(frac = 0.2)

train.head()

# Begin Training!

So, we have 14 columns remaining. Let us plot, for each feature, the samples against sale_price, to see their relative performance/their relevance.

In [ ]:
train.head() #from here, we have 72 different features to pick from and one label - sale_price
#The first feature we pick can be land_sqft

colnames = model_data_stepfour.columns.values
feature_colnames = colnames[:len(colnames)-1]
label_colname = colnames[len(colnames)-1]

results = pd.DataFrame(columns = colnames)
results['measurements'] = ['Coeff',"MSE","Variance","LinearScore"]
results.set_index('measurements',inplace=True)

fig = plt.figure(figsize=(8,15), facecolor='w', edgecolor='k')
fignum = 1;

for att in feature_colnames:
    #for att in feature_colnames:
    train_X1 = np.reshape(np.array(train[att]),[len(train),1])
    valid_X1 = np.reshape(np.array(valid[att]),[len(valid),1])

    train_Y = np.reshape(np.array(train['sale_price']),[len(train),1])
    valid_Y = np.reshape(np.array(valid['sale_price']),[len(valid),1])

    #create Linear Model
    regr = linear_model.LinearRegression(normalize=True)
    #fit Linear Model
    regr.fit(train_X1, train_Y)

    # Make predictions using the testing set
    pred_Y = regr.predict(test_X1)
    
    results[att].loc['Coeff'] = regr.coef_
    results[att].loc['MSE'] = mean_squared_error(test_Y, pred_Y)
    results[att].loc['Variance'] = r2_score(test_Y, pred_Y)
    results[att].loc['LinearScore'] = regr.score(test_Y, pred_Y)
    
    ax = fig.add_subplot(7,2,fignum)
    ax.scatter(test_X1, test_Y,  color='red', s = 3)
    ax.plot(test_X1, pred_Y, color='blue', linewidth = 1)
    ax.tick_params(length=2, width=1)
    ax.xaxis.set_major_locator(plt.MaxNLocator(3))
    ax.set_title(att)
    plt.tight_layout()
    fignum = fignum + 1

    
plt.show()
results.head()

Based on the graphs above (and through trial and error), we decide to remove certain columns, that end up improving the r^2 value.

In [ ]:

new_train = train.drop(columns = ['BsmtCode','AssessTot','block','AssessLand','BldgFront','BldgDepth'])
new_valid = valid.drop(columns = ['BsmtCode','AssessTot','block','AssessLand','BldgFront','BldgDepth'])


Now, we will try a number of different models with our final dataset, and find their respective R2 scores for the training and test dataset. In this way, we can find our most successful technique. For this, we have to import a number of models, then run our data through each model, and finally plot the R2 value for the test data to show its performance against other models.

In [ ]:
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.linear_model import Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import ensemble

Xtrn = new_train[new_train.columns.values[:-1]]
Ytrn = new_train[label_colname]
Xtest = new_test[new_valid.columns.values[:-1]]
Ytest = new_test[label_colname]

models = [LinearRegression(),
          linear_model.Lasso(alpha=0.1),
          Ridge(alpha=100.0),
          RandomForestRegressor(n_estimators=100, max_features='sqrt'),
          KNeighborsRegressor(n_neighbors=10),
          DecisionTreeRegressor(max_depth=4),
          ensemble.GradientBoostingRegressor()]

TestModels = pd.DataFrame()
tmp = {}

for model in models:
    print(model)
    m = str(model)
    tmp['Model'] = m[:m.index('(')] #get the chars in the string until char '('. Smart.
    model.fit(Xtrn, Ytrn)
    tmp['R2_Price'] = r2_score(Ytest, model.predict(Xtest))
    print('score on training',model.score(Xtrn, Ytrn))
    print('r2 score',r2_score(Ytest, model.predict(Xtest)))
    TestModels = TestModels.append([tmp])
    
TestModels.set_index('Model', inplace=True)

fig, axes = plt.subplots(ncols=1, figsize=(10, 4))
TestModels.R2_Price.plot(ax=axes, kind='bar', title='R2_Price')
plt.show()

TestModels


As we can see from the graph and table above, the best R2 value comes from the RandomForestRegressor. There are minimal signs of overfitting as the training and test scores are both fairly close.

# Discussion

The poorest performing models are the Linear, Lasso and Ridge regression models. The Lasso and Ridge regression models are quite similar to Linear, but penalise features that have little or no impact on the final prediction. As we have already manually removed features that do not add value, it is not surprising to see that all the regression models have similar performance.

KNeigboursRegressor works best at an optimum of about 10 nearest neigbours. Inherently, this model performs poorly on datasets with only a few points, and that too on predictions that involve extrapolation. As this is not the case, the model performs decently but does not capture all the intricacies of the data, and perhaps a weighted approach would work better.

GradientBoostingRegressor works quite well, though it is based off a decision tree, and between as a Random Forest is merely a more complicated and larger version of a decision tree, it is understandable why a Random Forest would provide a better approach than a singular decision tree. 

There are still improvements in accuracy that could be made, with various trial and error appraoches or based on previous heuristics. However, the current result was deemed suitable for the dataset and techniques used.